<a href="https://colab.research.google.com/github/Samarthsri1608/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests
!pip install html5lib
!pip install bs4
!pip install pdfplumber
!pip install google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 98.4 MB/s eta 0:00:00


In [8]:
import requests
import pdfplumber
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
from google import genai

In [4]:
pdf_url = 'https://vidyaprabodhinicollege.edu.in/VPCCECM/ebooks/ENGLISH%20LITERATURE/Harry%20potter/(Book%203)%20Harry%20Potter%20And%20The%20Prisoner%20Of%20Azkaban_001.pdf'


In [5]:
response = requests.get(pdf_url)
with open('document.pdf', 'wb') as f:
    f.write(response.content)

In [51]:
full_text = ""
with pdfplumber.open("document.pdf") as pdf:
  for page in pdf.pages:
    full_text += page.extract_text()

In [46]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [52]:
chunks = full_text.split("\n")
embeddings = embedder.encode(chunks)

In [53]:
def retrieve(query, top_n=3):
    query_vector = embedder.encode([query])
    sims = cosine_similarity(query_vector, embeddings)[0]
    top_idx = np.argsort(sims)[-top_n:][::-1]
    return [chunks[i] for i in top_idx]


In [54]:
GEMINI_API_KEY = "<<API KEY>>"
client = genai.Client(api_key=GEMINI_API_KEY)

In [55]:
def respond(query):
  Context = "\n".join(retrieve(query, top_n=3))
  print(f"Recieved Context = {Context}")
  response = client.models.generate_content(
      model="gemini-2.0-flash-lite", contents=f"""You are an assistant, answer the query based on the given context.
      Context: {Context}, query = {query}
      """
  )
  return response.text

In [64]:
print(respond("<<query>>"))

Recieved Context = Hermione, but not as though he were annoyed with her. He seemed to be
with him. For some reason this seemed to annoy Hermione as well; she
He turned to Hermione.
Hermione seemed to be annoyed, possibly because of the situation with "him" (whoever that is).

